In [7]:
import datetime
from xml.etree import ElementTree as ET
import requests
from requests.auth import HTTPDigestAuth
import numpy as np
import time
import json

In [12]:
# When power is negative it means it's consuming from the grid. Positive is backfeeding
class SonnenInterface():

    def __init__ (self, serial = None, auth_token = None):
        self.serial = serial
        self.token = auth_token
        self.url_ini = 'https://core-api.sonnenbatterie.de/proxy/'
        self.headers = { 'Accept': 'application/vnd.sonnenbatterie.api.core.v1+json',
    'Authorization': 'Bearer '+self.token,}
        
    
    def get_status(self):
        status_endpoint = '/api/v1/status'
        try:
            resp = requests.get(self.url_ini + self.serial + status_endpoint, headers=self.headers)

        except:
            print(exc)
            
        return resp.json()
    
    
# Backup:
# Intended to maintain an energy reserve for situations where the Grid is no longer available. During the off-grid 
# period the energy would be dispensed to supply the demand of power from all the essential loads. 
# Load management can be enabled to further extend the life of the batteries by the Developers.

    def enable_backup_mode(self):
        backup_endpoint = '/api/setting?EM_OperatingMode=7'
        try:
            resp = requests.get(self.url_ini + self.serial + backup_endpoint, headers=self.headers)
        
        except:
            print(exc)
            
        return resp.json()


# Self-Consumption:
# The ecoLinx monitors all energy sources (Grid, PV, Generator), loads, and Energy Reserve Percentage 
# in order to minimize purchase of energy from the Grid.

    def enable_self_consumption(self):
        sc_endpoint = '/api/setting?EM_OperatingMode=8'
        try:
            resp = requests.get(self.url_ini + self.serial + sc_endpoint, headers=self.headers)
            
        except:
            print(exc)
        
        return resp.json()
        

# Manual Mode
# This mode allows the user to manually charge or discharge the batteries. The user needs to provide the 
# value for charging or discharging and based on the value, the ecoLinx system will charge until it reaches 
# 100% or discharge until it reaches 0% User SOC (unless stopped by the user by changing the charge/discharge 
# value to 0).

    # Enabled by default
    def enable_manual_mode(self):
        manual_endpoint = '/api/setting?EM_OperatingMode=1'
        try:
            resp = requests.get(self.url_ini + self.serial + manual_endpoint, headers=self.headers)
            
        except:
            print(exc)
        
        return resp.json()
    
    
    def manual_mode_control(self, mode = 'charge', value = '0'):
        control_endpoint = '/api/v1/setpoint/'
        try:
            resp = requests.get(self.url_ini + self.serial + control_endpoint+mode+'/'+value, headers=self.headers)
            
        except:
            print(exc)
        
        return resp.json()
        
        

In [19]:
# Testing: define a serial and token

# token = 
# serial = 
SonnenInterface(serial = serial, auth_token = token).get_status()

{'Consumption_W': 0,
 'Fac': 60,
 'GridFeedIn_W': 0,
 'IsSystemInstalled': 1,
 'Production_W': 854,
 'RSOC': 91,
 'Timestamp': '2019-05-23 14:21:41',
 'USOC': 91,
 'Uac': 242,
 'Ubat': 53}

In [18]:
mode = 'discharge'
value = '500'
SonnenInterface(serial = '67682', auth_token = token).manual_mode_control(mode, value)

{'ReturnCode': '0'}